In [169]:
import pandas as pd
import re

import gspread
from google.oauth2.service_account import Credentials

import numpy as np
import time

### 1. Сведение ведомостей из ПДФ в таблицы

In [3]:
df = pd.read_excel('Ведомости_перечета_деревьев_старое.xlsx', header=None)

In [4]:
df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,квартал № 375 делянка № ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Ступень толщины, см","Число деревьев по породам, шт.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Модельные деревья для определения разряда высот,NaN,NaN,NaN,NaN,Число единичных и групповых семенных деревьев ...,NaN,NaN
2,NaN,Ель,NaN,NaN,Береза,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,деловой,полуделовой,дровяной,деловой,полуделовой,дровяной,деловой,полуделовой,дровяной,...,NaN,NaN,порода,"диаметр, см","высота, м","средняя высота, м",разряд высот,"ступень толщины, см",сосна,ель
4,8,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,NaN,...,NaN,NaN,Е,16,12,12,5,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,NaN,NaN
2096,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,NaN,NaN
2097,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,NaN,NaN
2098,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64,NaN,NaN


In [5]:
indexes_of_form = list(df[df[0].str.contains('квартал', case=False, na=False)].index)

In [6]:
list_of_forms = []
for i in indexes_of_form:
    form_data = df[(df.index == i)][0].values[0]
    try:
        kvartal = int(re.search(r'квартал.*?№\s*(.*?)\s*делянка', form_data).group(1).strip().replace(',','.'))
    except:
        kvartal = int(re.search(r'квартал.*?№\s*(.*?)\s*выдел', form_data).group(1).strip().replace(',','.'))
    videl = int(re.search(r'выдел.*?№\s*(.*?)\s*экспл', form_data).group(1).strip().replace(',','.'))
    exp_s = float(re.search(r'экспл\. площадь\s*(.*?)\s*га', form_data).group(1).strip().replace('\xa0\xa0\xa0 га','').replace(',','.'))
    try:
        recalc_s = float(re.search(r'площадь перечета\s*(.*?)\s*га', form_data).group(1).strip().replace(',','.'))
    except:
        recalc_s = float(re.search(r'площадь перечета\s*(.*)', form_data).group(1).strip().replace(',','.'))



    try:
        next_i = indexes_of_form[indexes_of_form.index(i)+1]
        form_dataframe = (df[(df.index > i)&(df.index < next_i)])

        forms_dict = {'квартал': kvartal,
                      'выдел': videl,
                      'экспл. площадь': exp_s,
                      'площадь перечета': recalc_s,
                      'dataframe':form_dataframe}

        list_of_forms.append(forms_dict)
    except:
        next_i = 2099
        form_dataframe = (df[(df.index > i)&(df.index <= next_i)])
        



        forms_dict = {'квартал': kvartal,
                      'выдел': videl,
                      'экспл. площадь': exp_s,
                      'площадь перечета': recalc_s,
                      'dataframe':form_dataframe}

        list_of_forms.append(forms_dict)

In [7]:
list_of_forms

[{'квартал': 375,
  'выдел': 6,
  'экспл. площадь': 0.052,
  'площадь перечета': 0.052,
  'dataframe':                      0                               1            2   \
  1   Ступень толщины, см  Число деревьев по породам, шт.          NaN   
  2                   NaN                             Ель          NaN   
  3                   NaN                         деловой  полуделовой   
  4                     8                             NaN          NaN   
  5                    12                             NaN          NaN   
  6                    16                              18          NaN   
  7                    20                             NaN          NaN   
  8                    24                             NaN          NaN   
  9                    28                             NaN          NaN   
  10                   32                             NaN          NaN   
  11                   36                             NaN          NaN   
  12       

### 2. Алгоритм поиска по получившимся таблицам

In [8]:
def find_form(by, number):
    dff = []
    for forms in list_of_forms:
        if isinstance(by, list):
            if len(by) == 1:
                if isinstance(number, list):
                    if not len(number) == len(by):
                        print('Кол-во значений и критериев должно совпадать')
                        break
                    if forms[by[0]] == number[0]:
                        dff.append(forms['dataframe'])
                else:
                    if forms[by[0]] == number:
                        dff.append(forms['dataframe'])

            if len(by) > 1:
                if not isinstance(number, list):
                    print('Ошибка значения должны быть списком')
                    break
                if not len(number) == len(by):
                    print('Кол-во значений и критериев должно совпадать')
                    break
            if len(by) == 2:
                if forms[by[0]] == number[0] and forms[by[1]] == number[1]:
                    dff.append(forms['dataframe'])   
                    
            if len(by) == 3:
                if forms[by[0]] == number[0] and forms[by[1]] == number[1] and forms[by[2]] == number[2]:
                    dff.append(forms['dataframe'])      

            if len(by) == 4:
                if forms[by[0]] == number[0] and forms[by[1]] == number[1] and forms[by[2]] == number[2] and forms[by[3]] == number[3]:
                    dff.append(forms['dataframe'])  
        else:
            if forms[by] == number:
                dff.append(forms['dataframe'])

    if len(dff) == 1:
        return dff[0]
    else:
        return dff



In [9]:
dff = find_form(by=['квартал', 'выдел'], number=[375, 25])

In [10]:
dff

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
141,"Ступень толщины, см","Число деревьев по породам, шт.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Модельные деревья для определения разряда высот,NaN,NaN,NaN,NaN,Число единичных и групповых семенных деревьев ...,NaN,NaN
142,NaN,Ель,NaN,NaN,Береза,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,NaN,деловой,полуделовой,дровяной,деловой,полуделовой,дровяной,деловой,полуделовой,дровяной,...,NaN,NaN,порода,"диаметр, см","высота, м","средняя высота, м",разряд высот,"ступень толщины, см",сосна,ель
144,8,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,...,NaN,NaN,Е,18,14,13.8,5,8,NaN,NaN
145,12,NaN,NaN,14,22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,18,16,NaN,NaN,12,NaN,NaN
146,16,22,NaN,NaN,2,NaN,3,NaN,NaN,NaN,...,NaN,NaN,NaN,18,11,NaN,NaN,16,NaN,NaN
147,20,15,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,...,NaN,NaN,NaN,18,14,NaN,NaN,20,NaN,NaN
148,24,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Б,12,11,10.8,7,24,NaN,NaN
149,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12,13,NaN,NaN,28,NaN,NaN
150,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12,8,NaN,NaN,32,NaN,NaN


### 3. Копирование ведомостей внутри файла расчета

In [178]:
def copy_ved(i):
# Авторизация
# Установите области доступа
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    # Авторизация
    credentials = Credentials.from_service_account_file(
        'stately-block-412016-7c237c2f8285.json',
        scopes=SCOPES
    )
    client = gspread.authorize(credentials)

    # Идентификаторы и имена таблиц
    spreadsheetId = "124-NyxnzYld1ajfUx9qhpdzkWfWiCF_u_K6-vcTTTCw"  # Установите ID таблицы
    sheetName = "Лист1"   # Установите имя листа
    sheetName_copy = "Лист2"

    # Открываем таблицу и лист
    spreadsheet = client.open_by_key(spreadsheetId)
    sheet = spreadsheet.worksheet(sheetName)
    sheet_copy = spreadsheet.worksheet(sheetName_copy)

    # Получаем размер листа
    cell_count = sheet.row_count * sheet.col_count

    # Получаем диапазон данных
    data_range = sheet.get_all_values()

    # Найдите последний заполненный ряд
    last_row = len(data_range)

    # Определите новый диапазон для вставки данных ниже
    start_row = last_row + 1  # Оставим одну строку между старым и новым диапазоном

    # Создайте новый диапазон для вставки
    new_range = f"A{start_row}:N{start_row + last_row - 1}"

    search_text = "Ведомость пересчета деревьев, назначенных в рубку"
    star_copy_row = [i for i, row in enumerate(data_range) if search_text in row][-1]

    # Скопируйте данные в новый диапазон
    body = {
        "requests": [
            {
                "copyPaste": {
                    "source": {
                        "sheetId": sheet_copy.id,
                        "startRowIndex": 0,
                        "endRowIndex": 22,
                        "startColumnIndex": 0,
                        "endColumnIndex": sheet.col_count
                    },
                    "destination": {
                        "sheetId": sheet.id,
                        "startRowIndex": i+22,
                        "endRowIndex": i+22+22,
                        "startColumnIndex": 0,
                        "endColumnIndex": sheet.col_count
                    },
                    "pasteType": "PASTE_NORMAL"
                }
            }
        ]
    }

    # Выполните запрос к API
    spreadsheet.batch_update(body)

In [180]:
copy_ved(22)

In [ ]:
delta = 22

### 4. Логика внесения значений в расчетную ведомость

In [15]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Авторизация
credentials = Credentials.from_service_account_file(
    'stately-block-412016-7c237c2f8285.json',
    scopes=SCOPES
)
client = gspread.authorize(credentials)

# Идентификаторы и имена таблиц
spreadsheetId = "1e2mn7V7KhXq63v93H9MvucmdYQAlhJsOeEhziLBC_6w"  # Установите ID таблицы
sheetName = "Лист1"   # Установите имя листа

# Открываем таблицу и лист
spreadsheet = client.open_by_key(spreadsheetId)
sheet_forth = spreadsheet.worksheet(sheetName)

# Идентификаторы и имена таблиц
spreadsheetId = "124-NyxnzYld1ajfUx9qhpdzkWfWiCF_u_K6-vcTTTCw"  # Установите ID таблицы
sheetName = "Лист1"   # Установите имя листа

# Открываем таблицу и лист
spreadsheet = client.open_by_key(spreadsheetId)
sheet_one = spreadsheet.worksheet(sheetName)


# Идентификаторы и имена таблиц
spreadsheetId = "1j2OA-E-Qxe04HRP3ciHJvHO85vvC948xjbvTs8RsUPs"  # Установите ID таблицы
sheetName = "Лист1"   # Установите имя листа

# Открываем таблицу и лист
spreadsheet = client.open_by_key(spreadsheetId)
sheet_second = spreadsheet.worksheet(sheetName)



In [318]:
all_rec = sheet_forth.get_all_records()
schet = 0
print('1')
second_df = pd.DataFrame(sheet_second.get_all_records())
second_df['Площадь (га)'] = second_df['Площадь (га)']/10000

def extract_quarter(text):
    match = re.search(r'кв\.\s*(\d+)', text)
    if match:
        return match.group(1)
    return ''
print('2')
second_df['Выдел под звездочкой'] = second_df['Примечания:'].apply(lambda x: x if x=='' else re.search(r'выд\.\s*(\d+)\s*\)', x).group(1))
second_df['Квартал под звездочкой'] = second_df['Примечания:'].apply(lambda x: extract_quarter(x))
second_df['Квартал под звездочкой'] = second_df.apply(lambda x: x['Номер квартала'] if x['Квартал под звездочкой'] == '' else x['Квартал под звездочкой'], axis=1)

second_df['Выдел под звездочкой'] = second_df['Выдел под звездочкой'].apply(lambda x: x if x == '' else int(x))
second_df['Квартал под звездочкой'] = second_df['Квартал под звездочкой'].apply(lambda x: x if x == '' else int(x))
print('3')

for record in all_rec[56:58]:
    actual_line = record
    if schet == 0:
        i = 0
    else:
        i+=22
    print (actual_line)


    ### апдейт квартала, выдела, площади
    kvartal_line = actual_line['Квартал']
    videl = actual_line['Выдел']
    videl_star = actual_line['Выдел*']
    area = actual_line['Площадь']/1000


    sheet_one.update(f'B{6+i}', [[kvartal_line]]) 
    sheet_one.update(f'D{6+i}', [[videl]])
    sheet_one.update(f'F{6+i}', [[area]])
    
    ### апдейт доп площадь


    try:
        second_area = second_df[(second_df['Квартал под звездочкой'] == kvartal_line) & (second_df['Выдел под звездочкой'] == videl_star)]['Площадь (га)'].values[0]
    except:
        try:
            second_area = second_df[(second_df['Квартал под звездочкой'] == kvartal_line) & (second_df['Номер выдела'] == videl)]['Площадь (га)'].values[0]
        except:
            try:
                second_area = second_df[(second_df['Номер квартала'] == kvartal_line) & (second_df['Выдел под звездочкой'] == videl_star)]['Площадь (га)'].values[0]
            except:
                second_area = second_df[(second_df['Номер квартала'] == kvartal_line) & (second_df['Номер выдела'] == videl)]['Площадь (га)'].values[0]
    sheet_one.update(f'G{6+i}', [[second_area]])

    ### апдейт таблиц дерева

    df_les = find_form(by=['экспл. площадь'], number=[second_area])
    koef = float(sheet_one.get_values('H6')[0][0].replace(',','.'))
    
    if len(df_les)>2:
        vidi = df_les[1:2].values.tolist()[0]
        vidi.remove('ВСЕГО')
        vidi = [x for x in vidi if not pd.isna(x)]
    
    if len(vidi) == 1:

        el_les_list = [item for sublist in sheet_one.get_values(f'C{9+i}:C{14+i}') for item in sublist]

        beg_list_1 = [9,10,11,12,13,14]
        beg_list_1 = [k+i for k in beg_list_1]

        if len(df_les)>2:
            for d_el, b in zip(el_les_list, beg_list_1):
                delovaya = df_les[df_les[0]==int(d_el)][1].values[0]*koef
                poludelova = df_les[df_les[0]==int(d_el)][2].values[0]*koef
                drovyanaya = df_les[df_les[0]==int(d_el)][3].values[0]*koef

                if not delovaya != delovaya:
                    delovaya = round(delovaya,0)
                    sheet_one.update(f'D{b}', [[delovaya]])
                if not poludelova != poludelova:
                    poludelova = round(poludelova,0)
                    sheet_one.update(f'E{b}', [[poludelova]])
                if not drovyanaya != drovyanaya:
                    drovyanaya = round(drovyanaya,0)
                    sheet_one.update(f'F{b}', [[drovyanaya]])        
        
            # ### Проверка наименований пород

        def is_number(s):
            try:
                float(s)
                return True
            except ValueError:
                return False
            
        if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{9+i}:H{14+i}') for item in sublist]):
            if vidi[0] == 'Береза':
                sheet_one.update(f'A{9+i}', [['Б']])
            if vidi[0] == 'Ель':
                sheet_one.update(f'A{9+i}', [['Е']])            

    if len(vidi) == 2:

        el_les_list = [item for sublist in sheet_one.get_values(f'C{9+i}:C{14+i}') for item in sublist]

        beg_list_1 = [9,10,11,12,13,14]
        beg_list_1 = [k+i for k in beg_list_1]

        if len(df_les)>2:
            for d_el, b in zip(el_les_list, beg_list_1):
                delovaya = df_les[df_les[0]==int(d_el)][1].values[0]*koef
                poludelova = df_les[df_les[0]==int(d_el)][2].values[0]*koef
                drovyanaya = df_les[df_les[0]==int(d_el)][3].values[0]*koef

                if not delovaya != delovaya:
                    delovaya = round(delovaya,0)
                    sheet_one.update(f'D{b}', [[delovaya]])
                if not poludelova != poludelova:
                    poludelova = round(poludelova,0)
                    sheet_one.update(f'E{b}', [[poludelova]])
                if not drovyanaya != drovyanaya:
                    drovyanaya = round(drovyanaya,0)
                    sheet_one.update(f'F{b}', [[drovyanaya]])        
        
            # ### Проверка наименований пород

        ### Ель

        el_les_list = [item for sublist in sheet_one.get_values(f'C{9+i}:C{14+i}') for item in sublist]

        beg_list_1 = [9,10,11,12,13,14]
        beg_list_1 = [k+i for k in beg_list_1]

        if len(df_les)>2:
            for d_el, b in zip(el_les_list, beg_list_1):
                delovaya = df_les[df_les[0]==int(d_el)][1].values[0]*koef
                poludelova = df_les[df_les[0]==int(d_el)][2].values[0]*koef
                drovyanaya = df_les[df_les[0]==int(d_el)][3].values[0]*koef

                if not delovaya != delovaya:
                    delovaya = round(delovaya,0)
                    sheet_one.update(f'D{b}', [[delovaya]])
                if not poludelova != poludelova:
                    poludelova = round(poludelova,0)
                    sheet_one.update(f'E{b}', [[poludelova]])
                if not drovyanaya != drovyanaya:
                    drovyanaya = round(drovyanaya,0)
                    sheet_one.update(f'F{b}', [[drovyanaya]])



        ### Береза

        bereza_les_list = [item for sublist in sheet_one.get_values(f'C{16+i}:C{20+i}') for item in sublist]

        beg_list_2 = [16,17,18,19,20]
        beg_list_2 = [k+i for k in beg_list_2]

        if len(df_les)>2:
            for d_bereza, b in zip(bereza_les_list, beg_list_2):
                delovaya = df_les[df_les[0]==int(d_bereza)][4].values[0]*koef
                poludelova = df_les[df_les[0]==int(d_bereza)][5].values[0]*koef
                drovyanaya = df_les[df_les[0]==int(d_bereza)][6].values[0]*koef

                if not delovaya != delovaya:
                    delovaya = round(delovaya,0)
                    sheet_one.update(f'D{b}', [[delovaya]])
                if not poludelova != poludelova:
                    poludelova = round(poludelova,0)
                    sheet_one.update(f'E{b}', [[poludelova]])
                if not drovyanaya != drovyanaya:
                    drovyanaya = round(drovyanaya,0)
                    sheet_one.update(f'F{b}', [[drovyanaya]])
        

        def is_number(s):
            try:
                float(s)
                return True
            except ValueError:
                return False
            
        if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{9+i}:H{14+i}') for item in sublist]):
            if vidi[0] == 'Береза':
                sheet_one.update(f'A{9+i}', [['Б']])
            if vidi[0] == 'Ель':
                sheet_one.update(f'A{9+i}', [['Е']])

        if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{16+i}:H{20+i}') for item in sublist]):
            if vidi[1] == 'Береза':
                sheet_one.update(f'A{16+i}', [['Б']])
            if vidi[1] == 'Ель':
                sheet_one.update(f'A{16+i}', [['Е']])




    # ### Ель

    # el_les_list = [item for sublist in sheet_one.get_values(f'C{9+i}:C{14+i}') for item in sublist]

    # beg_list_1 = [9,10,11,12,13,14]
    # beg_list_1 = [k+i for k in beg_list_1]

    # if len(df_les)>2:
    #     for d_el, b in zip(el_les_list, beg_list_1):
    #         delovaya = df_les[df_les[0]==int(d_el)][1].values[0]*koef
    #         poludelova = df_les[df_les[0]==int(d_el)][2].values[0]*koef
    #         drovyanaya = df_les[df_les[0]==int(d_el)][3].values[0]*koef

    #         if not delovaya != delovaya:
    #             delovaya = round(delovaya,0)
    #             sheet_one.update(f'D{b}', [[delovaya]])
    #         if not poludelova != poludelova:
    #             poludelova = round(poludelova,0)
    #             sheet_one.update(f'E{b}', [[poludelova]])
    #         if not drovyanaya != drovyanaya:
    #             drovyanaya = round(drovyanaya,0)
    #             sheet_one.update(f'F{b}', [[drovyanaya]])



    # ### Береза

    # bereza_les_list = [item for sublist in sheet_one.get_values(f'C{16+i}:C{20+i}') for item in sublist]

    # beg_list_2 = [16,17,18,19,20]
    # beg_list_2 = [k+i for k in beg_list_2]

    # if len(df_les)>2:
    #     for d_bereza, b in zip(bereza_les_list, beg_list_2):
    #         delovaya = df_les[df_les[0]==int(d_bereza)][4].values[0]*koef
    #         poludelova = df_les[df_les[0]==int(d_bereza)][5].values[0]*koef
    #         drovyanaya = df_les[df_les[0]==int(d_bereza)][6].values[0]*koef

    #         if not delovaya != delovaya:
    #             delovaya = round(delovaya,0)
    #             sheet_one.update(f'D{b}', [[delovaya]])
    #         if not poludelova != poludelova:
    #             poludelova = round(poludelova,0)
    #             sheet_one.update(f'E{b}', [[poludelova]])
    #         if not drovyanaya != drovyanaya:
    #             drovyanaya = round(drovyanaya,0)
    #             sheet_one.update(f'F{b}', [[drovyanaya]])


    # # ### Проверка наименований пород

    # def is_number(s):
    #     try:
    #         float(s)
    #         return True
    #     except ValueError:
    #         return False
        
    # if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{9+i}:H{14+i}') for item in sublist]):
    #     sheet_one.update(f'A{9+i}', [['Е']])
        
    # if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{16+i}:H{20+i}') for item in sublist]):
    #     sheet_one.update(f'A{16+i}', [['Б']])
    
    # ### Проверка наименований пород для боковой таблицы

    # if sheet_one.get_values(f'A{9+i}')[0]:
    #     if sheet_one.get_values(f'A{9+i}')[0][0] == 'Е':
    #         sheet_one.update(f'K{9+i}', [['Е']])
    # else:
    #     continue

    # if sheet_one.get_values(f'A{16+i}')[0]:
    #     if sheet_one.get_values(f'A{16+i}')[0][0] == 'Б':
    #         sheet_one.update(f'K{10+i}', [['Б']])
    # else:
    #     continue

    ### Вписывание значений в боковоую таблицу
    
    if len(vidi) == 1:
        if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{9+i}:H{14+i}') for item in sublist]):

            diametr_eli = df_les[df_les[18]==vidi[0][0]][19].values[0]
            visota_eli = df_les[df_les[18]==vidi[0][0]][21].values[0]
            razryad_eli = df_les[df_les[18]==vidi[0][0]][22].values[0]

            sheet_one.update(f'L{9+i}', [[diametr_eli]])
            sheet_one.update(f'M{9+i}', [[visota_eli]])
            sheet_one.update(f'N{9+i}', [[razryad_eli]])


    if len(vidi) == 2:
        if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{9+i}:H{14+i}') for item in sublist]):

            diametr_eli = df_les[df_les[18]==vidi[0][0]][19].values[0]
            visota_eli = df_les[df_les[18]==vidi[0][0]][21].values[0]
            razryad_eli = df_les[df_les[18]==vidi[0][0]][22].values[0]

            sheet_one.update(f'L{9+i}', [[diametr_eli]])
            sheet_one.update(f'M{9+i}', [[visota_eli]])
            sheet_one.update(f'N{9+i}', [[razryad_eli]])

        if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values(f'D{16+i}:H{20+i}') for item in sublist]):

            diametr_berez = df_les[df_les[18]==vidi[1][0]][19].values[0]
            visota_berez = df_les[df_les[18]==vidi[1][0]][21].values[0]
            razryad_berez = df_les[df_les[18]==vidi[1][0]][22].values[0]

            sheet_one.update(f'L{10+i}', [[diametr_berez]])
            sheet_one.update(f'M{10+i}', [[visota_berez]])
            sheet_one.update(f'N{10+i}', [[razryad_berez]])

    # Обновление таблицы 2

    diametr_do_16 = sum([int(item) for sublist in sheet_one.get_values(f'I{9+i}:I{11+i}') for item in sublist])+sum([int(item) for sublist in sheet_one.get_values(f'I{16+i}:I{18+i}') for item in sublist])
    diametr_over_16 = sum([int(item) for sublist in sheet_one.get_values(f'I{12+i}:I{14+i}') for item in sublist])+sum([int(item) for sublist in sheet_one.get_values(f'I{19+i}:I{20+i}') for item in sublist])

    df_forth = pd.DataFrame(sheet_forth.get_all_records())
    actual_index_in_forth = df_forth[(df_forth['Квартал'] == kvartal_line) & (df_forth['Выдел'] == videl)& (df_forth['Выдел*'] == videl_star)].index[0]+2

    sheet_forth.update(f'E{actual_index_in_forth}', [[diametr_do_16]])
    sheet_forth.update(f'F{actual_index_in_forth}', [[diametr_over_16]])

    # Обновление таблицы 2 еще одно
    elki_number = []
    bereza_number = []
    if sheet_one.get_values(f'A{9+i}')[0] and sheet_one.get_values(f'A{9+i}')[0][0] == 'Б':
        bereza_number.append(int(sheet_one.get_values(f'I{15+i}')[0][0]))
    if sheet_one.get_values(f'A{9+i}')[0] and sheet_one.get_values(f'A{9+i}')[0][0] == 'Е':
        elki_number.append(int(sheet_one.get_values(f'I{15+i}')[0][0]))
    if sheet_one.get_values(f'A{16+i}')[0] and sheet_one.get_values(f'A{16+i}')[0][0] == 'Б':
        bereza_number.append(int(sheet_one.get_values(f'I{21+i}')[0][0]))
    if sheet_one.get_values(f'A{16+i}')[0] and sheet_one.get_values(f'A{16+i}')[0][0] == 'Е':
        elki_number.append(int(sheet_one.get_values(f'I{21+i}')[0][0]))

    elki = sum(elki_number)
    bereza = sum(bereza_number)

    sheet_forth.update(f'J{actual_index_in_forth}', [[elki]])
    sheet_forth.update(f'K{actual_index_in_forth}', [[bereza]])

    copy_ved(i)
    
    schet+=1

    time.sleep(60)

1
2
3
{'Квартал': 374, 'Выдел': 9, 'Выдел*': 9, 'Площадь': 158, 'до 16 см': '', 'до 28 см': '', 'до 32 см': '', 'более 32 см': '', 'Итого:': 0, 'Е': '', 'Б': '', 'Ос': '', 'Л': '', 'Объем': 111}


/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_67170/1139489708.py:37: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'B{6+i}', [[kvartal_line]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_67170/1139489708.py:38: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'D{6+i}', [[videl]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_67170/1139489708.py:39: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'F{6+i}', [[area]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_67170/1139489708.py:54: DeprecationWarning: The or

{'Квартал': 374, 'Выдел': 2, 'Выдел*': 9, 'Площадь': 652, 'до 16 см': '', 'до 28 см': '', 'до 32 см': '', 'более 32 см': '', 'Итого:': 0, 'Е': '', 'Б': '', 'Ос': '', 'Л': '', 'Объем': 456}


In [317]:
all_rec[56:58]

[{'Квартал': 374,
  'Выдел': 9,
  'Выдел*': 9,
  'Площадь': 158,
  'до 16 см': '',
  'до 28 см': '',
  'до 32 см': '',
  'более 32 см': '',
  'Итого:': 0,
  'Е': '',
  'Б': '',
  'Ос': '',
  'Л': '',
  'Объем': 111},
 {'Квартал': 374,
  'Выдел': 2,
  'Выдел*': 9,
  'Площадь': 652,
  'до 16 см': '',
  'до 28 см': '',
  'до 32 см': '',
  'более 32 см': '',
  'Итого:': 0,
  'Е': '',
  'Б': '',
  'Ос': '',
  'Л': '',
  'Объем': 456}]

In [437]:
### апдейт квартала, выдела, площади

actual_line = sheet_forth.get_all_records()[14]

kvartal_line = actual_line['Квартал']
videl = actual_line['Выдел']
area = actual_line['Площадь']/1000

sheet_one.update('B6', [[kvartal_line]]) 
sheet_one.update('D6', [[videl]])
sheet_one.update('F6', [[area]])

In [473]:
### апдейт доп площад

second_df = pd.DataFrame(sheet_second.get_all_records())
second_df['Площадь (га)'] = second_df['Площадь (га)']/10000

second_area = second_df[(second_df['Номер квартала'] == kvartal_line) & (second_df['Номер выдела'] == videl)]['Площадь (га)'].values[0]

sheet_one.update('G6', [[second_area]])


/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/785083643.py:8: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update('G6', [[second_area]])


{'spreadsheetId': '124-NyxnzYld1ajfUx9qhpdzkWfWiCF_u_K6-vcTTTCw',
 'updatedRange': "'Лист1'!G6",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [481]:
### апдейт таблиц дерева

df_les = find_form(by=['экспл. площадь'], number=[second_area])

In [572]:
koef = float(sheet_one.get_values('H6')[0][0].replace(',','.'))

In [573]:
### Ель

el_les_list = [item for sublist in sheet_one.get_values('C9:C14') for item in sublist]


beg_list_1 = [9,10,11,12,13,14]
for d_el, i in zip(el_les_list, beg_list_1):
    delovaya = df_les[df_les[0]==int(d_el)][1].values[0]*koef
    poludelova = df_les[df_les[0]==int(d_el)][2].values[0]*koef
    drovyanaya = df_les[df_les[0]==int(d_el)][3].values[0]*koef

    if not delovaya != delovaya:
        sheet_one.update(f'D{i}', [[delovaya]])
    if not poludelova != poludelova:
        sheet_one.update(f'E{i}', [[poludelova]])
    if not drovyanaya != drovyanaya:
        sheet_one.update(f'F{i}', [[drovyanaya]])

    

/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/1873653720.py:13: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'D{i}', [[delovaya]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/1873653720.py:17: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'F{i}', [[drovyanaya]])


In [650]:
### Береза

bereza_les_list = [item for sublist in sheet_one.get_values('C16:C20') for item in sublist]

beg_list_2 = [16,17,18,19,20]
for d_bereza, i in zip(bereza_les_list, beg_list_2):
    delovaya = df_les[df_les[0]==int(d_bereza)][4].values[0]*koef
    poludelova = df_les[df_les[0]==int(d_bereza)][5].values[0]*koef
    drovyanaya = df_les[df_les[0]==int(d_bereza)][6].values[0]*koef

    if not delovaya != delovaya:
        sheet_one.update(f'D{i}', [[delovaya]])
    if not poludelova != poludelova:
        sheet_one.update(f'E{i}', [[poludelova]])
    if not drovyanaya != drovyanaya:
        sheet_one.update(f'F{i}', [[drovyanaya]])


/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/584575095.py:12: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'D{i}', [[delovaya]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/584575095.py:16: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'F{i}', [[drovyanaya]])


In [582]:
### Проверка наименований пород

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values('D9:I14') for item in sublist]):
    sheet_one.update(f'A9', [['E']])
    
if any(isinstance(item, (int, float)) or is_number(item) for item in [item for sublist in sheet_one.get_values('D16:I20') for item in sublist]):
    sheet_one.update(f'A16', [['Б']])

/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/1730322592.py:11: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'A9', [['E']])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/1730322592.py:14: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'A16', [['Б']])


In [586]:
### Проверка наименований пород для боковой таблицы

if sheet_one.get_values('A9')[0][0] == 'Е':
    sheet_one.update(f'K9', [['Е']])
if sheet_one.get_values('A16')[0][0] == 'Б':
    sheet_one.update(f'K10', [['Б']])

/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/696979975.py:6: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'K10', [['Б']])


In [609]:
### Вписывание значений в боковоую таблицу

diametr_eli = df_les[df_les[18]=='Е'][19].values[0]
visota_eli = df_les[df_les[18]=='Е'][21].values[0]
razryad_eli = df_les[df_les[18]=='Е'][22].values[0]

sheet_one.update(f'L9', [[diametr_eli]])
sheet_one.update(f'M9', [[visota_eli]])
sheet_one.update(f'N9', [[razryad_eli]])



diametr_berez = df_les[df_les[18]=='Б'][19].values[0]
visota_berez = df_les[df_les[18]=='Б'][21].values[0]
razryad_berez = df_les[df_les[18]=='Б'][22].values[0]

sheet_one.update(f'L10', [[diametr_berez]])
sheet_one.update(f'M10', [[visota_berez]])
sheet_one.update(f'N10', [[razryad_berez]])


/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3205810755.py:7: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'L9', [[diametr_eli]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3205810755.py:8: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'M9', [[visota_eli]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3205810755.py:9: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_one.update(f'N9', [[razryad_eli]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3205810755.py:17: DeprecationWarning: The order 

{'spreadsheetId': '124-NyxnzYld1ajfUx9qhpdzkWfWiCF_u_K6-vcTTTCw',
 'updatedRange': "'Лист1'!N10",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [651]:
# Обновление таблицы 2

diametr_do_16 = sum([int(item) for sublist in sheet_one.get_values('I9:I11') for item in sublist])+sum([int(item) for sublist in sheet_one.get_values('I16:I18') for item in sublist])
diametr_over_16 = sum([int(item) for sublist in sheet_one.get_values('I12:I14') for item in sublist])+sum([int(item) for sublist in sheet_one.get_values('I19:I20') for item in sublist])

df_forth = pd.DataFrame(sheet_forth.get_all_records())
actual_index_in_forth = df_forth[(df_forth['Квартал'] == kvartal_line) & (df_forth['Выдел'] == videl)].index[0]+2

sheet_forth.update(f'E{actual_index_in_forth}', [[diametr_do_16]])
sheet_forth.update(f'F{actual_index_in_forth}', [[diametr_over_16]])


/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3904281711.py:9: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_forth.update(f'E{actual_index_in_forth}', [[diametr_do_16]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3904281711.py:10: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_forth.update(f'F{actual_index_in_forth}', [[diametr_over_16]])


{'spreadsheetId': '1e2mn7V7KhXq63v93H9MvucmdYQAlhJsOeEhziLBC_6w',
 'updatedRange': "'Лист1'!F16",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [659]:
# Обновление таблицы 2 еще одно

elki = int(sheet_one.get_values('I15')[0][0])
bereza = int(sheet_one.get_values('I21')[0][0])

sheet_forth.update(f'J{actual_index_in_forth}', [[elki]])
sheet_forth.update(f'K{actual_index_in_forth}', [[bereza]])

/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3053518110.py:6: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_forth.update(f'J{actual_index_in_forth}', [[elki]])
/var/folders/n5/k72sy9791zx1gr065ybnk52w0000gn/T/ipykernel_61173/3053518110.py:7: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet_forth.update(f'K{actual_index_in_forth}', [[bereza]])


{'spreadsheetId': '1e2mn7V7KhXq63v93H9MvucmdYQAlhJsOeEhziLBC_6w',
 'updatedRange': "'Лист1'!K16",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}